### PROC步
----
#### 常用通用选项
VAR:规定用这个过程分析的一些变量；

WEIGHT:规定一个变量，它的值是这些观测的相应权数；

CLASS:在分析中指定一些变量为分类变量；

BY:规定一些变量，SAS过程对输入数据集用by变量定义的几个 数据组分别进行分析处理；

OUTPUT/OUT:给出用该过程产生的输出数据集的信息；

In [1]:
OPTIONS COMPRESS = YES;

DATA CARS;
SET SASHELP.CARS;
RUN;

SAS Connection established. Subprocess id is 11770



#### PROC SORT

(1)基本形式:

PROC SORT 
    DATA = 
    OUT = ; 
BY ; 
RUN;

SAS会按照第一个变量先排序，再对后面排序；

SAS默认排序是按升序排序，若需降序排序，在变量名前加DESCENDING；

(2)排序设置：

NODUPKEY、NOUNIQUEKEY、NODUP(探索);

In [2]:
/* PROC SORT */
PROC SORT DATA = SASHELP.CARS;
BY MSRP;
RUN;

In [3]:
PROC SORT DATA = SASHELP.CARS OUT = CARS_MSRP;
BY DESCENDING MSRP;
RUN;

PROC SORT DATA = SASHELP.CARS OUT = CARS_MAKE;
BY MAKE;
RUN;

In [15]:
DATA CARS_DUP;
KEEP TYPE ORIGIN DRIVETRAIN;
SET SASHELP.CARS;
RUN;

PROC PRINT DATA = CARS_DUP(OBS=20);
RUN;

Obs,Type,Origin,DriveTrain
1,SUV,Asia,All
2,Sedan,Asia,Front
3,Sedan,Asia,Front
4,Sedan,Asia,Front
5,Sedan,Asia,Front
6,Sedan,Asia,Front
7,Sports,Asia,Rear
8,Sedan,Europe,Front
9,Sedan,Europe,Front
10,Sedan,Europe,Front


In [16]:
PROC SORT DATA = CARS_DUP(OBS=20) OUT = CARS_NDK NODUPKEY;
BY TYPE ORIGIN;
RUN;

PROC PRINT DATA = CARS_NDK;
RUN;

Obs,Type,Origin,DriveTrain
1,SUV,Asia,All
2,Sedan,Asia,Front
3,Sedan,Europe,Front
4,Sports,Asia,Rear


In [17]:
PROC SORT DATA = CARS_DUP(OBS=20) OUT = CARS_NQK NOUNIQUEKEY;
BY TYPE ORIGIN;
RUN;

PROC PRINT DATA = CARS_NQK;
RUN;

Obs,Type,Origin,DriveTrain
1,Sedan,Asia,Front
2,Sedan,Asia,Front
3,Sedan,Asia,Front
4,Sedan,Asia,Front
5,Sedan,Asia,Front
6,Sedan,Europe,Front
7,Sedan,Europe,Front
8,Sedan,Europe,Front
9,Sedan,Europe,All
10,Sedan,Europe,All


In [18]:
PROC SORT DATA = CARS_DUP(OBS=20) OUT = CARS_ND NODUP;
BY TYPE ORIGIN;
RUN;

PROC PRINT DATA = CARS_ND;
RUN;

Obs,Type,Origin,DriveTrain
1,SUV,Asia,All
2,Sedan,Asia,Front
3,Sedan,Europe,Front
4,Sedan,Europe,All
5,Sedan,Europe,Front
6,Sedan,Europe,All
7,Sedan,Europe,Front
8,Sedan,Europe,All
9,Sports,Asia,Rear


#### PROC FREQ

(1)基本形式:

PROC FREQ DATA=;TABLE ;RUN; 

对一个变量计算频数，称作ONE-WAY；

对两个变量计算频数，称作TWP-WAY； 

对多个变量计算频数，称作交叉表；

(2)TABLE VAR后常接选项: 

NOCOL:不打印列百分比 

NOROW:不打印行百分比 

NOPERCENT:不打印百分比 

MISSING:统计缺失值

**/OUT:将频次表输出到数据集中**

(3)TABLE TYPE:

TABLE COL_A

TABLE COL_A * COL_B

TABLE (COL_A COL_C) * COL_B

TABLE COL_A * COL_B / NOROW NOCOL NOPERCENT

TABLE COL_A * COL_B / MISSING

In [19]:
/*	PROC FREQ*/
PROC FREQ DATA = CARS NOPRINT;
TABLE DRIVETRAIN / OUT = CARS_FREQ;
RUN;

PROC PRINT DATA = CARS_FREQ;
RUN;

Obs,DriveTrain,COUNT,PERCENT
1,All,92,21.4953
2,Front,226,52.8037
3,Rear,110,25.7009


In [20]:
PROC FREQ DATA = CARS;
TABLE DRIVETRAIN * ORIGIN/NOCOL NOROW NOPERCENT MISSING;
RUN;

In [21]:
PROC FREQ DATA = CARS;
TABLE ORIGIN * DRIVETRAIN/NOCOL NOROW NOPERCENT MISSING;
RUN;

In [22]:
PROC FREQ DATA = CARS;
TABLE DRIVETRAIN * ORIGIN;
WHERE TYPE = "SUV";
RUN;

In [26]:
PROC FREQ DATA = CARS(OBS=20);
TABLE TYPE * DRIVETRAIN * ORIGIN;
RUN;

In [27]:
PROC FREQ DATA = CARS;
TABLE (TYPE DRIVETRAIN) * ORIGIN;
RUN;

#### PROC UNIVARIATE

(1)基本形式:

PROC UNIVARIATE DATA = ;VAR ;RUN;

单变量过程，可以求单个变量的大部分统计指标

In [29]:
/* PROC UNIVARIATE */
PROC UNIVARIATE DATA = CARS;
VAR MSRP;
RUN;

In [32]:
PROC UNIVARIATE DATA = CARS;
VAR MAKE;
RUN;

In [41]:
PROC UNIVARIATE DATA = CARS(OBS=100);
VAR MSRP;
CLASS MAKE;
RUN;

#### PROC TRANSPOSE

(1)基本形式:

PROC TRANSPOSE DATA = OUT = ;VAR ;RUN;

实现对数据集的转置，即把观测变为变量，变量变为观测；


(2)常用选项:

PREFIX:规定转置后的变量名前缀；

SUFFIX:规定转置后的变量名后缀；

OUT:规定输出数据集；

ID:使用其后规定的变量值作为输出数据集中被转置的变量名；

VAR:规定需要转置的变量名；

BY:规定分组的变量名；


In [35]:
/* PROC TRANSPOSE */
DATA CARS_MAMS;
KEEP MAKE TYPE MSRP;
SET SASHELP.CARS;
RUN;

PROC SORT DATA = CARS_MAMS OUT = CARS_MAMSD NODUPKEY;
BY MAKE TYPE;
RUN;

PROC PRINT DATA = CARS_MAMSD;
RUN;

Obs,Make,Type,MSRP
1,Acura,SUV,"$36,945"
2,Acura,Sedan,"$23,820"
3,Acura,Sports,"$89,765"
4,Audi,Sedan,"$25,940"
5,Audi,Sports,"$84,600"
6,Audi,Wagon,"$40,840"
7,BMW,SUV,"$37,000"
8,BMW,Sedan,"$28,495"
9,BMW,Sports,"$48,195"
10,BMW,Wagon,"$32,845"


In [39]:
PROC TRANSPOSE DATA = CARS_MAMSD;
VAR MSRP;
BY MAKE;
RUN;

/* DATAX */

In [40]:
PROC TRANSPOSE DATA = CARS_MAMSD OUT = CARS_M PREFIX = ID_;
VAR MSRP;
BY MAKE;
ID TYPE;
RUN;

PROC PRINT DATA = CARS_M;
RUN;

Obs,Make,_NAME_,ID_SUV,ID_Sedan,ID_Sports,ID_Wagon,ID_Truck,ID_Hybrid
1,Acura,MSRP,"$36,945","$23,820","$89,765",.,.,.
2,Audi,MSRP,.,"$25,940","$84,600","$40,840",.,.
3,BMW,MSRP,"$37,000","$28,495","$48,195","$32,845",.,.
4,Buick,MSRP,"$37,895","$22,180",.,.,.,.
5,Cadillac,MSRP,"$52,795","$30,835","$76,200",.,"$52,975",.
6,Chevrolet,MSRP,"$42,735","$11,690","$44,535","$22,225","$36,100",.
7,Chrysler,MSRP,.,"$17,985","$34,495","$31,230",.,.
8,Dodge,MSRP,"$32,235","$13,670","$81,795",.,"$17,630",.
9,Ford,MSRP,"$41,475","$13,270","$18,345","$17,475","$22,010",.
10,GMC,MSRP,"$31,890","$25,640",.,.,"$16,530",.


#### PROC SURVEYSELECT

(1)基本形式:

PROC SURVEYSELECT DATA = METHOD = N = ;RUN;

PROC SURVEYSELECT DATA = METHOD = SAMPRATE = ;RUN;

实现对数据的随机抽样；


(2)常用选项:

METHOD：随机抽样的方法

**SRS（Simple Random Sampling，不放回简单随机抽样）；**

**URS（Unrestricted Random Sampling，放回简单随机抽样）；**

**SYS（Systematic Sampling，系统抽样）；**

SEED：随机种子数，随机数产生器；

非负整数，若为0则以当前时间作为当前随机中子数，则可实现每次抽取的样本不同；

若取大于0的整数，则下次抽样时若输入相同值即可得到相同的样本；


In [43]:
/* PROC SURVEYSELECT */
PROC SURVEYSELECT 
    DATA = CARS_MAMS METHOD = SRS N = 3 
    OUT = CARS_SRS_N3;
RUN;

PROC PRINT DATA = CARS_SRS_N3;
RUN;

In [46]:
PROC SURVEYSELECT 
    DATA = CARS_MAMS METHOD = SRS SAMPRATE = 0.01 
    OUT = CARS_SRS_P1;
RUN;

PROC PRINT DATA = CARS_SRS_P1;
RUN;

In [45]:
PROC SURVEYSELECT 
    DATA = CARS_MAMS METHOD = SRS SAMPRATE = 0.01 
    OUT = CARS_SRS_N1;
STRATA MAKE;
RUN;

PROC PRINT DATA = CARS_SRS_N1;
RUN;